In [1]:
# rest of imports
import numpy as np
from numpy import load, savez_compressed
from sklearn.decomposition import IncrementalPCA
from os import listdir
import re
from cuml.decomposition import IncrementalPCA as IncrementalPCA_CUML
import cupy as cp
import cupyx

In [2]:
# LFW_IPCA Class Definition
class LFW_IncrementalPCA(IncrementalPCA):
    def __init__(self, dataset_paths: list[str], n_components: int, batch_size: int = 16):
        super(LFW_IncrementalPCA, self).__init__(
            n_components=n_components,
            batch_size = batch_size
        )
        self.dataset_paths = dataset_paths

    def start_fit(self, display_progress: bool=False):
        r""" Begins an incremental fit on the given dataset paths
            Args:
                display_progress (bool): Whether to display incremental progress of the fit

            Returns:
                None
        """

        for i, file in enumerate(self.dataset_paths):
            data = load(file)
            X_train = data['arr_0']
            X_train = X_train.reshape(X_train.shape[0], -1)
            if (display_progress):
                print(f"i = {i} | Partial fit on X_train of sizes {X_train.shape}")
            self.partial_fit(X_train)

    def get_embeddings(self, display_progress: bool=False):
        r""" Transforms the given datasets into embeddings and returns them
            Args:
                display_progress (bool): Whether to display final progress of embeddings

            Returns:
                X_train_pca (np.array): X_train pca embeddings
                Y_train_pca (np.array): Y_train labels
                X_test_pca (np.array): X_test pca embeddings
                Y_test_pca (np.array): Y_test labels
        """

        X_train_pca = X_test_pca = np.zeros((0, self.n_components_))
        Y_train_pca = Y_test_pca = np.zeros((0, 1))

        for i, file in enumerate(self.dataset_paths):
            data = load(file)
            X_train, Y_train, X_test, Y_test = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
            X_train = X_train.reshape(X_train.shape[0], -1)
            X_test = X_test.reshape(X_test.shape[0], -1)
            Y_train = Y_train.reshape(-1,1)
            Y_test = Y_test.reshape(-1,1)

            X_train_pca = np.vstack((X_train_pca, self.transform(X_train)))
            X_test_pca = np.vstack((X_test_pca, self.transform(X_test)))

            Y_train_pca = np.vstack((Y_train_pca, Y_train))
            Y_test_pca = np.vstack((Y_test_pca, Y_test))

        if display_progress:
            print(f"X_train_pca shape {X_train_pca.shape} | X_test_pca {X_test_pca.shape} | Y_train_pca shape {Y_train_pca.shape} | Y_test_pca {Y_test_pca.shape}")
        return X_train_pca, Y_train_pca, X_test_pca, Y_test_pca

    def save_embeddings(self, save_path: str, display_progress: bool=False):
        r""" Transforms the given datasets into embeddings and returns them
            Args:
            save_path (str): npz file path to save the embeddings to
                display_progress (bool): Whether to display final progress of embeddings

            Returns:
                None
        """

        X_train_pca = X_test_pca = np.zeros((0, self.n_components_))
        Y_train_pca = Y_test_pca = np.zeros((0, 1))

        for i, file in enumerate(self.dataset_paths):
            data = load(file)
            X_train, Y_train, X_test, Y_test = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
            X_train = X_train.reshape(X_train.shape[0], -1)
            X_test = X_test.reshape(X_test.shape[0], -1)
            Y_train = Y_train.reshape(-1,1)
            Y_test = Y_test.reshape(-1,1)

            X_train_pca = np.vstack((X_train_pca, self.transform(X_train)))
            X_test_pca = np.vstack((X_test_pca, self.transform(X_test)))

            Y_train_pca = np.vstack((Y_train_pca, Y_train))
            Y_test_pca = np.vstack((Y_test_pca, Y_test))

        if display_progress:
            print(f"X_train_pca shape {X_train_pca.shape} | X_test_pca {X_test_pca.shape} | Y_train_pca shape {Y_train_pca.shape} | Y_test_pca {Y_test_pca.shape}")

        savez_compressed(save_path, X_train_pca, Y_train_pca, X_test_pca, Y_test_pca)

In [8]:
# LFW_IPCA Class Definition
class LFW_IncrementalPCA_CUML(IncrementalPCA_CUML):
    def __init__(self, *args, dataset_paths: list[str] = [], n_components: int, batch_size: int = 16):
        super(LFW_IncrementalPCA_CUML, self).__init__(
            args,
            n_components=n_components,
            batch_size = batch_size
        )
        self.dataset_paths = dataset_paths

    def start_fit(self, display_progress: bool=False):
        r""" Begins an incremental fit on the given dataset paths
            Args:
                display_progress (bool): Whether to display incremental progress of the fit

            Returns:
                None
        """

        for i, file in enumerate(self.dataset_paths):
            data = load(file)
            X_train = data['arr_0']
            X_train = X_train.reshape(X_train.shape[0], -1).astype('float32')

            # X_train_gpu = cp.asarray(X_train)
            if (display_progress):
                print(f"i = {i} | Partial fit on X_train of sizes {X_train.shape}")
            self.partial_fit(X_train)
            # del X_train_gpu
            # cp._default_memory_pool.free_all_blocks()

    def get_embeddings(self, display_progress: bool=False):
        r""" Transforms the given datasets into embeddings and returns them
            Args:
                display_progress (bool): Whether to display final progress of embeddings

            Returns:
                X_train_pca (np.array): X_train pca embeddings
                Y_train_pca (np.array): Y_train labels
                X_test_pca (np.array): X_test pca embeddings
                Y_test_pca (np.array): Y_test labels
        """

        X_train_pca = X_test_pca = np.zeros((0, self.n_components_))
        Y_train_pca = Y_test_pca = np.zeros((0, 1))

        for i, file in enumerate(self.dataset_paths):
            data = load(file)
            X_train, Y_train, X_test, Y_test = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
            X_train = X_train.reshape(X_train.shape[0], -1).astype('float32')
            X_test = X_test.reshape(X_test.shape[0], -1).astype('float32')
            Y_train = Y_train.reshape(-1,1)
            Y_test = Y_test.reshape(-1,1)

            X_train_pca = np.vstack((X_train_pca, self.transform(X_train)))
            X_test_pca = np.vstack((X_test_pca, self.transform(X_test)))

            Y_train_pca = np.vstack((Y_train_pca, Y_train))
            Y_test_pca = np.vstack((Y_test_pca, Y_test))

        if display_progress:
            print(f"X_train_pca shape {X_train_pca.shape} | X_test_pca {X_test_pca.shape} | Y_train_pca shape {Y_train_pca.shape} | Y_test_pca {Y_test_pca.shape}")
        return X_train_pca, Y_train_pca, X_test_pca, Y_test_pca

    def save_embeddings(self, save_path: str, display_progress: bool=False):
        r""" Transforms the given datasets into embeddings and returns them
            Args:
            save_path (str): npz file path to save the embeddings to
                display_progress (bool): Whether to display final progress of embeddings

            Returns:
                None
        """

        X_train_pca = X_test_pca = np.zeros((0, self.n_components_))
        Y_train_pca = Y_test_pca = np.zeros((0, 1))

        for i, file in enumerate(self.dataset_paths):
            data = load(file)
            X_train, Y_train, X_test, Y_test = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
            X_train = X_train.reshape(X_train.shape[0], -1).astype('float32')
            X_test = X_test.reshape(X_test.shape[0], -1).astype('float32')
            Y_train = Y_train.reshape(-1,1)
            Y_test = Y_test.reshape(-1,1)

            X_train_pca = np.vstack((X_train_pca, cp.asnumpy(self.transform(cp.asarray(X_train)))))
            X_test_pca = np.vstack((X_test_pca, cp.asnumpy(self.transform(cp.asarray(X_test)))))

            Y_train_pca = np.vstack((Y_train_pca, Y_train))
            Y_test_pca = np.vstack((Y_test_pca, Y_test))

        if display_progress:
            print(f"X_train_pca shape {X_train_pca.shape} | X_test_pca {X_test_pca.shape} | Y_train_pca shape {Y_train_pca.shape} | Y_test_pca {Y_test_pca.shape}")

        savez_compressed(save_path, X_train_pca, Y_train_pca, X_test_pca, Y_test_pca)

In [9]:
import re
from time import time
cp._default_memory_pool.free_all_blocks()
n_components = 150
dataset_npz_filenames = []
reg_expr = '^lfw-deepfunneled-dataset-cuml_[0-9]{4}.npz'
for file in listdir('./'):
    if re.search(reg_expr, file):
        dataset_npz_filenames.append(file)

# start = time()
# # create an lfw_ipca instance and give the label of datasets and desired n_componentes
# lfw_ipca = LFW_IncrementalPCA_CUML(dataset_paths=dataset_npz_filenames, n_components=n_components)

# # start the fit on the entire datasets given
# lfw_ipca.start_fit(display_progress=True)

# # save the embeddings as a compressed .npz file
# lfw_ipca.save_embeddings("lfw-deepfunneled-pca-embeddings-150-cuml.npz", display_progress=True)
# end = time()
# print(f"Time for cuml method to complete {end - start}")

start = time()
# create an lfw_ipca instance and give the label of datasets and desired n_componentes
lfw_ipca = LFW_IncrementalPCA(dataset_npz_filenames, n_components=n_components)

# start the fit on the entire datasets given
lfw_ipca.start_fit(display_progress=True)

# save the embeddings as a compressed .npz file
lfw_ipca.save_embeddings("lfw-deepfunneled-pca-embeddings-150-default.npz", display_progress=True)
end = time()
print(f"Time for default method to complete {end - start}")

/tmp/ipykernel_49644/2433422669.py:4: FutureWarning: Pass handle=() as keyword args. From version 21.06, passing these as positional arguments will result in an error
  super(LFW_IncrementalPCA_CUML, self).__init__(


i = 0 | Partial fit on X_train of sizes (348, 76800)
i = 1 | Partial fit on X_train of sizes (328, 76800)
i = 2 | Partial fit on X_train of sizes (599, 76800)
i = 3 | Partial fit on X_train of sizes (271, 76800)
i = 4 | Partial fit on X_train of sizes (356, 76800)
i = 5 | Partial fit on X_train of sizes (824, 76800)
i = 6 | Partial fit on X_train of sizes (422, 76800)
i = 7 | Partial fit on X_train of sizes (319, 76800)
i = 8 | Partial fit on X_train of sizes (298, 76800)
i = 9 | Partial fit on X_train of sizes (325, 76800)
i = 10 | Partial fit on X_train of sizes (470, 76800)
i = 11 | Partial fit on X_train of sizes (341, 76800)
i = 12 | Partial fit on X_train of sizes (328, 76800)
i = 13 | Partial fit on X_train of sizes (244, 76800)
i = 14 | Partial fit on X_train of sizes (425, 76800)
i = 15 | Partial fit on X_train of sizes (460, 76800)
i = 16 | Partial fit on X_train of sizes (309, 76800)
i = 17 | Partial fit on X_train of sizes (339, 76800)
i = 18 | Partial fit on X_train of siz